In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
#  Create Synthetic Dataset
np.random.seed(42)
n = 200

In [ ]:
data = {
    "video_watch_time": np.random.randint(10, 300, n),
    "quiz_scores_avg": np.random.randint(0, 100, n),
    "forum_activity": np.random.randint(0, 50, n),
    "assignments_submitted": np.random.randint(0, 10, n),
}

df = pd.DataFrame(data)

# Dropout rule for synthetic target
df["dropout"] = (
    (df["video_watch_time"] < 60) |
    (df["quiz_scores_avg"] < 40) |
    (df["assignments_submitted"] < 3)
).astype(int)
df.head()

,video_watch_time,quiz_scores_avg,forum_activity,assignments_submitted,dropout
0,112,81,18,4,0
1,280,0,27,5,1
2,116,10,25,2,1
3,81,91,36,7,0
4,198,56,25,0,1


In [ ]:
# Features and target
X = df.drop("dropout", axis=1)
y = df["dropout"]

# Keep feature names consistent
feature_names = X.columns

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X[feature_names], y, test_size=0.2, random_state=42
)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on test set
X_test_df = pd.DataFrame(X_test, columns=feature_names)
y_pred = model.predict(X_test_df)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        14
           1       0.92      0.92      0.92        26

    accuracy                           0.90        40
   macro avg       0.89      0.89      0.89        40
weighted avg       0.90      0.90      0.90        40



In [ ]:
# Function for new predictions
def predict_dropout(video_watch_time, quiz_scores_avg, forum_activity, assignments_submitted):
    input_df = pd.DataFrame(
        [[video_watch_time, quiz_scores_avg, forum_activity, assignments_submitted]],
        columns=feature_names
    )
    prob = model.predict_proba(input_df)[0][1]  # probability of dropout
    prediction = model.predict(input_df)[0]
    return prediction, round(prob*100, 2)



In [ ]:
import ipywidgets as widgets
widgets.IntSlider()
from IPython.display import display, clear_output

In [ ]:
video_watch_time = widgets.IntSlider(description="Video Time(minutes) :", min=0, max=300, value=60,style={'description_width': '130px'})
quiz_scores_avg = widgets.IntSlider(description="Quiz Avg % :", min=0, max=100, value=50,style={'description_width': '78px'})
forum_activity = widgets.IntSlider(description="Forum Posts :", min=0, max=50, value=5)
assignments_submitted = widgets.IntSlider(description="Assignments :", min=0, max=10, value=2)

button = widgets.Button(description="Predict Dropout", button_style='warning')

In [ ]:
output = widgets.Output()

# Button click callback
def on_button_click(b):
    with output:
        output.clear_output(wait=True)
        pred, prob = predict_dropout(
            video_watch_time.value,
            quiz_scores_avg.value,
            forum_activity.value,
            assignments_submitted.value
        )
        status = "❌ High Dropout Risk" if pred == 1 else "✅ Low Dropout Risk"
        print(f"{status} (Probability: {prob}%)")

button.on_click(on_button_click)

# Heading
heading = widgets.HTML(value="<h2 style='color:black; font-weight:bold;'>Online Course Dropout Risk Predictor</h2>")

# Display UI with heading
ui = widgets.VBox([
    heading,
    video_watch_time,
    quiz_scores_avg,
    forum_activity,
    assignments_submitted,
    button,
    output
])
display(ui)